In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgbm
from lightgbm import *

既に多くのBaseline、EDAが有志の方々によって挙げられていたため、そこは省略します。
今回は初めてのkaggleということで、特徴量を１つ作成した上でLGBMを利用した予測を行いました。

investment_numはtime_idごとのinvestment_idの数の合計を当てはめたもので、investment_idの数が多い時期ほど投資の背景に何らかの事象が発生していると考えられ、予測に活用できる、との指摘がありました。
https://qiita.com/tmrtj9999/items/15abb2ba3a6ea124dcbc
これを参考に特徴量を作成しました。本家ではdatesetの作り方までは公開されていなかったので、独自にやり方を再現しました。


当コンペでは、使えるRAMがUbiquantデータセットを読み込めるほどの大きさではありません。そこで、何らかの形でファイルを圧縮する必要があります。
ここではpickleという形式を使い、データを圧縮します。作り方は簡単で、df.to_pickleを利用するだけです。
データにはsupplemental_train.csvが含まれており、これは追加のデータです。このデータと先述の特徴量を加えたpklを独自に作成しました。
データ圧縮の方法はもう一つあり、その場合はint,float型を64から下げることです。その場合はdef reduce_mamory_usageなどを作成してください。

In [ ]:
df = pd.read_pickle("../input/low-mem-investment/train_num_investment.pkl")

In [ ]:
df.head()

In [ ]:
a = df.columns
print(a)

b = df["target"].max()
print(b)

In [ ]:
from sklearn.model_selection import KFold, train_test_split


features = [f'f_{i}' for i in range(300)] + ["num_investment"]
target = 'target'
 

df_features = df[features]


X_train, X_val, Y_train, Y_val = train_test_split(df_features, df[target], train_size=0.80, shuffle=False)

del df

df = [[]]
df_features = [[]]

paramの値はランダムです。ベイズ最適化など、もっと良い方法を利用できると思います。

In [ ]:
import warnings
import numpy as np
import lightgbm as lgb
from scipy.stats import pearsonr

warnings.simplefilter('ignore')

lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval = lgb.Dataset(X_val, Y_val, reference=lgb_train)

params = {
        "objective": "regression",
        "num_leaves": 14,
        "max_depth": 4,
        "metric" : "rmse",
        "feature_fraction": 0.8,
        "subsample_freq": 1,
        "bagging_fraction": 0.7,
        "min_data_in_leaf": 10,
        "learning_rate": 0.2,
        "boosting": "gbdt",
        "lambda_l1": 0.4,
        "lambda_l2": 0.4,
        "random_state": 42,
}
        
        
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round =3000,
                early_stopping_rounds=3
                )

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    
    
    
    test_df.drop(['row_id'], axis=1, inplace=True)
    test_df.drop(['investment_id'], axis=1, inplace=True)
    test_df['num_investment'] = len(test_df)
    pred = gbm.predict(test_df)
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df)   # register predictions